# 🎭 Political Emotion Detection Analysis
## Advanced Emotion Classification for Nepali Political Discourse

### Research Objectives:
- **RQ1**: How can emotion detection enhance campaign planning strategies?
- **RQ2**: Compare effectiveness of emotion detection vs traditional sentiment analysis
- **RQ3**: Identify emotional bias and ethical challenges in political discourse

### Emotion Categories:
- **Primary Emotions**: Anger, Fear, Joy, Sadness, Trust, Anticipation, Disgust, Surprise
- **Political Context**: Power, Authority, Rebellion, Hope, Despair, Unity, Division

---

**Author**: Your Name  
**Date**: August 2025  
**Project**: Data Driven Analysis of Political Sentiment Analysis

## 1. Import Required Libraries
Essential libraries for emotion detection, data processing, and visualization

In [ ]:
# Core Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# NLP and Transformers
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.nn.functional import softmax

# Data Processing
import re
import emoji
import json
from collections import Counter, defaultdict
from datetime import datetime, timedelta

# Visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import wordcloud
from wordcloud import WordCloud

# Twitter API
import tweepy

# Statistical Analysis
from scipy import stats
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

print("✅ All libraries imported successfully!")
print(f"🔥 PyTorch version: {torch.__version__}")
print(f"🤗 Transformers available")
print(f"📊 Plotly available for interactive visualizations")

## 2. Load and Configure Emotion Detection Models
Initialize multiple emotion detection models for comparative analysis

In [ ]:
# Initialize multiple emotion detection models for research comparison
class EmotionDetectionSuite:
    def __init__(self):
        self.models = {}
        self.load_all_models()
    
    def load_all_models(self):
        """Load different emotion detection models for comparative analysis"""
        
        print("🔄 Loading emotion detection models...")
        
        # Model 1: General emotion detection
        try:
            self.models['emotion_general'] = pipeline(
                "text-classification", 
                model="j-hartmann/emotion-english-distilroberta-base",
                tokenizer="j-hartmann/emotion-english-distilroberta-base"
            )
            print("✅ General emotion model loaded")
        except Exception as e:
            print(f"❌ Failed to load general emotion model: {e}")
        
        # Model 2: Twitter-specific emotion detection
        try:
            self.models['emotion_twitter'] = pipeline(
                "text-classification",
                model="cardiffnlp/twitter-roberta-base-emotion",
                tokenizer="cardiffnlp/twitter-roberta-base-emotion"
            )
            print("✅ Twitter emotion model loaded")
        except Exception as e:
            print(f"❌ Failed to load Twitter emotion model: {e}")
        
        # Model 3: Load existing sentiment model for comparison
        try:
            self.models['sentiment_nepali'] = pipeline(
                "text-classification", 
                model="nepali-sentiment-model-xlmr", 
                tokenizer="nepali-sentiment-model-xlmr"
            )
            print("✅ Nepali sentiment model loaded")
        except Exception as e:
            print(f"❌ Failed to load Nepali sentiment model: {e}")
            self.models['sentiment_nepali'] = None
    
    def predict_emotions(self, text, model_name='emotion_general'):
        """Predict emotions for given text using specified model"""
        if model_name not in self.models or self.models[model_name] is None:
            return {"error": f"Model {model_name} not available"}
        
        try:
            results = self.models[model_name](text)
            return results
        except Exception as e:
            return {"error": str(e)}
    
    def get_available_models(self):
        """Return list of successfully loaded models"""
        return [name for name, model in self.models.items() if model is not None]

# Initialize the emotion detection suite
emotion_suite = EmotionDetectionSuite()
print(f"\n🎯 Available models: {emotion_suite.get_available_models()}")

# Define emotion categories for analysis
EMOTION_CATEGORIES = {
    'basic_emotions': ['anger', 'fear', 'joy', 'sadness', 'surprise', 'disgust'],
    'political_emotions': ['anger', 'fear', 'joy', 'sadness', 'trust', 'anticipation'],
    'sentiment_mapping': {
        'anger': 'negative',
        'fear': 'negative', 
        'sadness': 'negative',
        'disgust': 'negative',
        'joy': 'positive',
        'trust': 'positive',
        'anticipation': 'neutral',
        'surprise': 'neutral'
    }
}

print(f"\n📊 Emotion categories defined: {EMOTION_CATEGORIES['basic_emotions']}")

## 3. Data Preprocessing for Emotion Analysis
Enhanced preprocessing functions designed specifically for emotion detection

In [ ]:
# Enhanced preprocessing for emotion detection
class EmotionPreprocessor:
    def __init__(self):
        # Load Nepali stopwords
        try:
            with open("nepali_stopwords.txt", "r", encoding="utf-8") as f:
                self.nepali_stopwords = set(f.read().splitlines())
            print("✅ Nepali stopwords loaded")
        except FileNotFoundError:
            print("⚠️ Creating basic Nepali stopwords")
            self.nepali_stopwords = {'र', 'को', 'मा', 'छ', 'हो', 'ले', 'लाई', 'भन्न', 'गर्न'}
        
        # Emotion-preserving emoji mapping
        self.emotion_emojis = {
            'anger': ['😠', '😡', '🤬', '👿', '💢'],
            'joy': ['😊', '😄', '😃', '🙂', '😁', '🤗', '🎉'],
            'sadness': ['😢', '😭', '😞', '😔', '💔', '😿'],
            'fear': ['😨', '😰', '😱', '😳', '🙀'],
            'surprise': ['😮', '😯', '😲', '🤯', '😦'],
            'disgust': ['🤢', '🤮', '😷', '🤧']
        }
    
    def extract_emotion_indicators(self, text):
        """Extract emotion indicators from text"""
        indicators = {
            'emojis': [],
            'exclamations': 0,
            'questions': 0,
            'caps_ratio': 0,
            'repetitions': 0
        }
        
        # Extract emojis
        indicators['emojis'] = [char for char in text if char in emoji.EMOJI_DATA]
        
        # Count exclamations and questions
        indicators['exclamations'] = text.count('!')
        indicators['questions'] = text.count('?')
        
        # Calculate caps ratio
        if len(text) > 0:
            indicators['caps_ratio'] = sum(1 for c in text if c.isupper()) / len(text)
        
        # Detect repetitions (like "noooo" or "yessss")
        repetition_pattern = r'(.)\1{2,}'
        indicators['repetitions'] = len(re.findall(repetition_pattern, text.lower()))
        
        return indicators
    
    def clean_for_emotion_analysis(self, text):
        """Clean text while preserving emotional context"""
        if pd.isnull(text):
            return ""
        
        original_text = text
        
        # Extract emotion indicators before cleaning
        emotion_indicators = self.extract_emotion_indicators(text)
        
        # Convert emojis to text descriptions (preserve emotional meaning)
        text = emoji.demojize(text, delimiters=(" ", " "))
        
        # Remove URLs but keep emotional punctuation
        text = re.sub(r"http\S+|www\S+|https\S+", "", text)
        
        # Remove handles but keep emotional context
        text = re.sub(r"@\w+", "", text)
        
        # Clean special characters but preserve emotional punctuation
        text = re.sub(r"[\u200c\u200d\u200e\u200f\u202a-\u202e]", "", text)
        
        # Remove hashtags but keep the content
        text = re.sub(r"#(\w+)", r"\1", text)
        
        # Preserve emotional emphasis (multiple punctuation)
        text = re.sub(r"([!?]){2,}", r"\1\1", text)  # Keep double for emphasis
        
        # Remove excessive whitespace
        text = re.sub(r"\s+", " ", text).strip()
        
        # Filter stopwords but keep emotional words
        emotional_words = {'राम्रो', 'नराम्रो', 'डर', 'खुशी', 'दुःख', 'रिस', 'चिन्ता'}
        words = text.split()
        filtered_words = []
        
        for word in words:
            if (len(word) >= 2 and 
                (word not in self.nepali_stopwords or word in emotional_words)):
                filtered_words.append(word)
        
        cleaned_text = " ".join(filtered_words)
        
        return {
            'cleaned_text': cleaned_text,
            'original_text': original_text,
            'emotion_indicators': emotion_indicators,
            'length': len(cleaned_text),
            'word_count': len(filtered_words)
        }
    
    def batch_preprocess(self, texts):
        """Process multiple texts efficiently"""
        results = []
        for text in texts:
            processed = self.clean_for_emotion_analysis(text)
            results.append(processed)
        return results

# Initialize preprocessor
emotion_preprocessor = EmotionPreprocessor()

# Test preprocessing with sample political texts
sample_texts = [
    "सरकारको भ्रष्टाचारले जनतामा निराशा फैलिएको छ! 😠",
    "नयाँ संविधानले देशमा शान्ति ल्याउने आशा छ 🙏✨",
    "राजनीतिक दलहरूको वचन कहिले पूरा हुन्छ??? 😞💔",
    "चुनावमा सबै उम्मेदवारले जनताको भावना बुझ्नुपर्छ"
]

print("🧪 Testing emotion preprocessing:")
for i, text in enumerate(sample_texts):
    result = emotion_preprocessor.clean_for_emotion_analysis(text)
    print(f"\n{i+1}. Original: {text}")
    print(f"   Cleaned: {result['cleaned_text']}")
    print(f"   Indicators: {result['emotion_indicators']}")
    print(f"   Stats: {result['word_count']} words, {result['length']} chars")